In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from matplotlib import dates
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from IPython.display import HTML
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Function definitions
def roundPartial(value, resolution, decimals=4):
    return np.around(np.round(value / resolution) * resolution, decimals=decimals)

def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val

In [ ]:
# Read in the radar data
radar_name = 'KGWX'
radar_type= 'NEXRAD'
elev_angle = 0.5
elev_str = str(elev_angle).replace('.', 'p')

# For 03/30/22 case (PERiLS-2022 IOP2)
date = '0330'
radar_start_datetimestamp = '20220330220424'
radar_end_datetimestamp = '20220331035633'
height = 500.

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

radar_basedir = \
    '/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/IOP2/KGWX'
#radar_basedir = os.path.join(radar_basedir, '{}/{}'.format(date, radar_name[1:]))
radar_input_dir = os.path.join(radar_basedir, 'extracted_sweeps')
radar_output_dir = os.path.join(radar_basedir, 'extracted_sweeps')
if not os.path.exists(radar_output_dir):
    os.makedirs(radar_output_dir)

radar_start_timestamp = datetime_start.strftime('%Y%m%d%H%M')
radar_end_timestamp = datetime_end.strftime('%Y%m%d%H%M')

radar_paths = glob.glob(radar_input_dir + f'/{radar_name}*el{elev_str}.nc')

radar_paths = sorted(radar_paths)
radar_input_list = []

for path in radar_paths:
    filename = os.path.basename(path)
    file_timestamp = filename[4:19]
    # print(file_timestamp)
    file_datetime = datetime.strptime(file_timestamp, '%Y%m%d_%H%M%S')
    if file_datetime >= datetime_start and file_datetime <= datetime_end:
        radar_input_list.append(path)

# Read in the individual sweeps
radarobj_list = []
for radar_path in radar_input_list:
    print(f"Reading {os.path.basename(radar_path)}")
    radarobj = pyart.io.read(radar_path)
    radarobj_list.append(radarobj)

In [ ]:
ZH_name = 'REF'
ZDR_name = 'ZDR'
RHV_name = 'RHO'

med_filter_width = 5

RHV_thresh = 0.95
dBZ_thresh = 5.0

for radar_path, radarobj in zip(radar_input_list, radarobj_list):
    
    # Make copies of polarimetric fields
    for field_name in [ZH_name, ZDR_name, RHV_name]:
        radarobj.add_field_like(field_name, field_name + '_filtered',
                                radarobj.fields[field_name]['data'].copy(), replace_existing=True)

    print("Applying median filter")
    footprint = [[1, 1, 1, 1, 1]]    
    for field_name in [ZH_name, ZDR_name, RHV_name]:
        radarobj.fields[field_name + '_filtered']['data'] = \
            ndimage.median_filter(radarobj.fields[field_name + '_filtered']['data'].copy(),
                                  footprint=footprint)

    print("Creating dBZ and RHV gate filter")
    # Create a gate filter to mask out areas with dBZ and RHV below thresholds
    rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radarobj,
                                                              rhv_field=RHV_name + '_filtered',
                                                              refl_field=ZH_name + '_filtered',
                                                              min_ncp=None,
                                                              min_rhv=RHV_thresh,
                                                              min_refl=dBZ_thresh,
                                                              max_refl=None)

    print("Applying gate filter")
    # Mask fields using the dBZ/RHV mask
    for field_name in [ZH_name, ZDR_name, RHV_name]:
        radarobj.fields[field_name + '_filtered']['data'] = \
            np.ma.masked_where(rhoHV_ref_filter.gate_excluded,
                               radarobj.fields[field_name + '_filtered']['data'])
    
    orig_filename = os.path.basename(radar_path)
    new_filename = orig_filename.replace('.nc', '_filt.nc')
    new_filepath = os.path.join(radar_output_dir, new_filename)
    print(f"Saving {new_filename}...")
    pyart.io.write_cfradial(new_filepath, radarobj)